# Vulnerability Ireland

TO DO:
- change census names to 10 characters
- allow the domains and dimensions to be weighted differently

## Script Setup

In [38]:
#load the libraries
library(sf)
library(rgdal)
library(rgeos)
library(raster)
library(tidyverse)
library(RColorBrewer)

In [39]:
#set a directory for export 
exportDirectory <- "../2_OutputData"

# Get the data

In [60]:
#census data
censusData <- read.csv("../1_InputData/1a_CensusData/censusDataZ.csv", header=TRUE, sep=",", stringsAsFactors = FALSE)
# head(censusData)

#impervious surface
imperviousSurfaceData <- read.csv("../1_InputData/1c_ImperviousSurface/imperviousSurface.csv", header=TRUE, sep=",", stringsAsFactors = FALSE)
imperviousSurfaceData <- imperviousSurfaceData %>% select('SA_GUID__1', 'impervious')
# head(imperviousSurfaceData)

#vulnerabiltiy combine info to help guide the amalgamation of the data - weighting should be changed in this file.
combineInfo <- read.csv("../0_Scripts/VulnerabilityIndicators/vulnerabilityIndicatorsInformation.csv", header=TRUE, sep=",", stringsAsFactors = FALSE, fileEncoding="UTF-8-BOM")
#replace spaces in column names with _
names(combineInfo) <- gsub('\\.', '_', names(combineInfo))
combineInfo


domain,indicator,sens,prepare,respond,recover,enhExp,indVulRel,weighting
age,young,1,0,0,0,0,1,0.5000000
age,old,1,0,0,0,0,1,0.5000000
health,poorHealth,1,0,0,0,0,1,0.5000000
health,disability,1,0,0,0,0,1,0.5000000
income,unemploy,0,1,1,1,0,1,0.2000000
income,lowSkill,0,1,1,1,0,1,0.2000000
income,farming,0,1,1,1,0,1,0.2000000
income,oneParent,0,1,1,1,0,1,0.2000000
income,familyUnits,0,1,1,1,0,1,0.2000000
info,noInternet,0,1,1,1,0,1,0.3333333


## Combine the data

In [61]:
#combine the data into one master dataset
masterDataset <- merge(censusData, imperviousSurfaceData, by = "SA_GUID__1")
head(masterDataset)

SA_GUID__1,young,old,priSch,volunteers,poorHealth,disability,unemploy,lowSkill,farming,...,yearBuilt,mobHome,unoccupiedDwellings,oneParent,onePerson,noCar,noInternet,priWater,familyUnits,impervious
000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,3.3881284,-0.8683376,1.7138802,-0.88629267,-0.72495811,-0.92237878,-0.61651676,-0.3191883,-0.5775567,...,-0.8371340,-0.1877975,-0.2345341,-0.9538197,-1.4596547236,-0.80432752,-0.6252661,-0.5390502,-0.28865195,-0.3062535
00275248-3d78-4fdb-8e5c-8ff1f76aebea,-1.2041833,1.0830533,-0.5610454,0.81756980,0.27802125,-0.62612269,-0.23523658,-0.7175526,0.2814339,...,-0.4334151,-0.1877975,-0.3976059,-0.5813873,-0.0009694625,-0.65730407,1.0787377,1.3836361,0.24855950,-0.4046186
002b83e4-5cce-414a-8570-871a50ae0964,0.2940145,-0.6564055,0.3585147,-0.52366291,0.07647707,0.06773966,0.22240936,-0.7371132,-0.1126784,...,-0.7250814,-0.1877975,-0.5907944,0.3083305,-0.2053648681,-0.07352003,0.1779948,-0.5390502,-0.06192227,0.2528440
005eec3f-d746-480f-ae2c-bc6f1151548d,-1.3337553,1.4547683,-1.3575178,0.27506223,0.49314740,0.45605479,-0.07660144,-1.9157874,-0.5775567,...,-0.7827719,-0.1877975,-0.3274759,-1.0919120,-0.6526063004,1.08000764,-1.2116383,-0.3106122,-1.18354741,0.5785988
00b00ae4-229d-455d-84f1-d6face4876b1,1.7131410,-1.2060597,2.9123835,-1.59443632,-0.32806286,-0.03079729,0.75770342,1.5272111,-0.5775567,...,-0.7424689,-0.1877975,-0.6509824,3.2417674,-1.1159816411,-0.02238398,-0.8409014,-0.5390502,1.26645393,0.5467843
00df2b78-66c8-4009-89e0-0e14e15e4208,1.5084518,-0.8589787,0.6262863,0.02987263,-1.03120608,-1.02733001,-0.97843629,0.4816620,-0.4599395,...,-0.6196854,-0.1877975,-0.4166131,-0.4959347,-0.9415930720,-0.83852272,-0.7636645,-0.5390502,-0.57203551,-0.4271954


# Indicator Correlation

Need to check if the indicators are too strongly correlated with each other (+-0.85).

In [62]:
masterDatasetCor <- masterDataset %>% select(-'SA_GUID__1')
indicatorCorrelation <- round(cor(masterDatasetCor), 2)
indicatorCorrelation

,young,old,priSch,volunteers,poorHealth,disability,unemploy,lowSkill,farming,rent,...,yearBuilt,mobHome,unoccupiedDwellings,oneParent,onePerson,noCar,noInternet,priWater,familyUnits,impervious
young,1.00,-0.32,0.58,-0.11,-0.20,-0.35,-0.13,0.01,-0.02,0.00,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.09
old,-0.32,1.00,-0.37,0.21,0.31,0.52,0.08,-0.02,0.11,-0.32,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.15
priSch,0.58,-0.37,1.00,-0.01,-0.23,-0.37,-0.14,0.10,0.05,-0.12,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.23
volunteers,-0.11,0.21,-0.01,1.00,-0.25,-0.13,-0.36,-0.21,0.40,-0.50,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.48
poorHealth,-0.20,0.31,-0.23,-0.25,1.00,0.64,0.57,0.29,-0.13,0.22,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.16
disability,-0.35,0.52,-0.37,-0.13,0.64,1.00,0.55,0.28,-0.12,0.13,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.10
unemploy,-0.13,0.08,-0.14,-0.36,0.57,0.55,1.00,0.44,-0.10,0.39,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.13
lowSkill,0.01,-0.02,0.10,-0.21,0.29,0.28,0.44,1.00,0.07,0.06,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.14
farming,-0.02,0.11,0.05,0.40,-0.13,-0.12,-0.10,0.07,1.00,-0.45,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,-0.58
rent,0.00,-0.32,-0.12,-0.50,0.22,0.13,0.39,0.06,-0.45,1.00,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0.59


# Data Processing

## Indicator Relationship

In [63]:
#get the indicator relationship with vulnerability (+1 or -1)
indRelationship <- combineInfo %>% select('indicator', 'indVulRel')
indRelationship <- indRelationship %>%
    spread(key = 'indicator', value = 'indVulRel')
indRelationship


disability,education,engLang,familyUnits,farming,impervious,lowSkill,mobHome,newRes,noCar,...,priSch,priWater,renewableEnergyHouses,rent,travelTime,unemploy,unoccupiedDwellings,volunteers,yearBuilt,young
1,1,1,1,1,1,1,1,1,1,...,-1,1,-1,1,1,1,1,-1,1,1


In [64]:
#multiply the indicators by their vulnerabiltiy relationship

#get the column names and weights
names <- names(indRelationship)
weights <- indRelationship[, names]

#copy and rename the dataset
indicatorRelationshipDataset <- masterDataset

#multiply the data
indicatorRelationshipDataset[, names] <- sweep(indicatorRelationshipDataset[, names], 2, unlist(weights[, names]), "*")
# indicatorRelationshipDataset


## Domains

In [65]:
# get the domains and their associated indicator ID
domainsIndicators <- combineInfo %>% select('domain', 'indicator')
domainsIndicators

domain,indicator
age,young
age,old
health,poorHealth
health,disability
income,unemploy
income,lowSkill
income,farming
income,oneParent
income,familyUnits
info,noInternet


In [66]:
#get a list of the unique domains
domains <- unique(domainsIndicators$domain)
domains

[1] "age"      "health"   "income"   "info"     "locKnow"  "mobility"
 [7] "access"   "socNet"   "tenure"   "physEnv"  "houChar"

In [67]:
#create the start of the dataset in which further data will be added with the GUID
domainDatasetMerge <- masterDataset %>% select('SA_GUID__1')

#loop through each the domains and:
for (domainI in domains){
    #identify which indicators are used within the domain
    domainData <- domainsIndicators %>% filter(domain == domainI)
    
    #get a vector of indicator IDs for the domain
    domainDataIDs <- domainData$indicator
#     print(domainDataIDs)
  
    #count the number of indicators in the domain
    indicatorCount <- length(domainData$indicator)
#     print(indicatorCount)
    
    #add the GUID column name to the vector
    domainDataIDs <- (c('SA_GUID__1', domainDataIDs))
#     str(domainDataIDs)
    
#     filter the master dataset to only the indicators in the domain
    masterDatasetDomainFiltered <- indicatorRelationshipDataset[domainDataIDs]
#    str(masterDatasetDomainFiltered)

    #calculate the weight of each indicator (assumed to be EQUAL)
    weighting <- 1/indicatorCount
#     print(weighting)
          
    #weight the data
    weightedData <- masterDatasetDomainFiltered %>% mutate_if(is.numeric, function(x) {x*weighting})
#     str(weightedData)
    
    
    #sum the data to get the total score for the domain
    weightedData[, domainI] <- rowSums(weightedData[2:(indicatorCount+1)], na.rm = TRUE)
    
    #add the total for the domain to the merging table
    domainDatasetMerge <- merge(domainDatasetMerge, weightedData %>% select('SA_GUID__1', domainI), by = 'SA_GUID__1')
    
}

#all the total domain scores collated into one table
head(domainDatasetMerge)


SA_GUID__1,age,health,income,info,locKnow,mobility,access,socNet,tenure,physEnv,houChar
000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,1.25989542,-0.82366845,-0.55114667,-0.3552211,0.6516841,-0.80432752,1.0782434,-0.76241407,-0.6291566,-0.3062535,-0.8488325
00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.06056501,-0.17405072,-0.20083663,-0.1481699,-0.2804696,-0.65730407,-0.4619628,-0.08583129,-1.0319586,-0.4046186,-0.1742794
002b83e4-5cce-414a-8570-871a50ae0964,-0.18119549,0.07210836,-0.07619481,0.3552176,3.1072677,-0.07352003,2.0566698,-0.01340555,0.3733692,0.2528440,-0.4831587
005eec3f-d746-480f-ae2c-bc6f1151548d,0.06050649,0.47460109,-0.96908098,-0.5608797,4.2239853,1.08000764,-0.7677756,0.14328309,1.1891490,0.5785988,-0.2048429
00b00ae4-229d-455d-84f1-d6face4876b1,0.25354066,-0.17943007,1.24311585,-0.6141689,-0.7106899,-0.02238398,-0.5841088,-0.81130961,2.1279860,0.5467843,-1.0339710
00df2b78-66c8-4009-89e0-0e14e15e4208,0.32473656,-1.02926804,-0.40493681,-0.4923913,0.9026496,-0.83852272,3.0769416,-0.53258399,-0.6391862,-0.4271954,-0.6258975


## Dimensions

Need to collate the domains into the five dimensions.

In [68]:
# create a vector of the dimension names
dimensions <- c('sens', 'prepare', 'respond', 'recover', 'enhExp')

#create the dataset to combine the results with just the GUID present
dimensionDatasetMerge <- masterDataset %>% select('SA_GUID__1')

# get the domain and dimension information
dimensionInfo <- combineInfo %>% select(c('domain', dimensions))
# dimensionInfo

# loop through each of the dimensions and:
for (dimensionI in dimensions){
    #get all of the domains and a single dimension
    dimensionIndicators <- dimensionInfo %>% select(c('domain', dimensionI))
    
    #filter the domains by the dimension - 1 means that the domain is part of the dimension
    dimensionIndicators <- dimensionIndicators %>% filter(dimensionIndicators[, dimensionI] == 1)
       
    #get a unique vector of the domains
    domainsDimensionIndicator <- unique(dimensionIndicators$domain)
    
    #get the number of domains that is in the dimension
    domainsDimensionIndicatorCount = length(domainsDimensionIndicator)
#     print(domainsDimensionIndicatorCount)
    
    #create a dataset with only the domains for the dimension
    dimensionTotal <- domainDatasetMerge %>% select(c('SA_GUID__1', domainsDimensionIndicator))  
    str(dimensionTotal)
    
#     #create the new dimension field name
#     dimensionName = paste(dimensionI)
    
    #get the sum of the dimensions
    dimensionTotal[, dimensionI] <- rowSums(dimensionTotal[2:(domainsDimensionIndicatorCount+1)], na.rm = TRUE)
#      str(dimensionTotal)
    
    #add the total for the dimension to the merging table
    dimensionDatasetMerge <- merge(dimensionDatasetMerge, dimensionTotal %>% select('SA_GUID__1', dimensionI), by = 'SA_GUID__1')   
    
}
head(dimensionDatasetMerge)



'data.frame':	18919 obs. of  3 variables:
 $ SA_GUID__1: chr  "000581a3-4ebd-4a74-b5f8-0bd78cd7ede5" "00275248-3d78-4fdb-8e5c-8ff1f76aebea" "002b83e4-5cce-414a-8570-871a50ae0964" "005eec3f-d746-480f-ae2c-bc6f1151548d" ...
 $ age       : num  1.2599 -0.0606 -0.1812 0.0605 0.2535 ...
 $ health    : num  -0.8237 -0.1741 0.0721 0.4746 -0.1794 ...
'data.frame':	18919 obs. of  5 variables:
 $ SA_GUID__1: chr  "000581a3-4ebd-4a74-b5f8-0bd78cd7ede5" "00275248-3d78-4fdb-8e5c-8ff1f76aebea" "002b83e4-5cce-414a-8570-871a50ae0964" "005eec3f-d746-480f-ae2c-bc6f1151548d" ...
 $ income    : num  -0.5511 -0.2008 -0.0762 -0.9691 1.2431 ...
 $ info      : num  -0.355 -0.148 0.355 -0.561 -0.614 ...
 $ locKnow   : num  0.652 -0.28 3.107 4.224 -0.711 ...
 $ tenure    : num  -0.629 -1.032 0.373 1.189 2.128 ...
'data.frame':	18919 obs. of  7 variables:
 $ SA_GUID__1: chr  "000581a3-4ebd-4a74-b5f8-0bd78cd7ede5" "00275248-3d78-4fdb-8e5c-8ff1f76aebea" "002b83e4-5cce-414a-8570-871a50ae0964" "005eec3f-d746-480f-ae

SA_GUID__1,sens,prepare,respond,recover,enhExp
000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.43622697,-0.8838402,-0.7431818,-2.4731094,-1.1550860
00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.23461573,-1.6614347,-1.8345743,-1.0921419,-0.5788979
002b83e4-5cce-414a-8570-871a50ae0964,-0.10908713,3.7596597,5.3560348,0.1920972,-0.2303147
005eec3f-d746-480f-ae2c-bc6f1151548d,0.53510759,3.8831736,3.1495397,-0.3066699,0.3737558
00b00ae4-229d-455d-84f1-d6face4876b1,0.07411059,2.0462432,-1.4995453,-0.2047466,-0.4871867
00df2b78-66c8-4009-89e0-0e14e15e4208,-0.70453148,-0.6338646,1.7111564,-2.2684348,-1.0530929


In [50]:
#standardise the data

#copy the data so that Z scores can be calculated
dimensionDatasetMergeZ <- dimensionDatasetMerge

#if the column is a numeric, then scale the data
dimensionDatasetMergeZ %>% mutate_if(is.numeric, scale)

head(dimensionDatasetMergeZ)

SA_GUID__1,sens,prepare,respond,recover,enhExp
000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.38290364,-0.65162272,-0.49006099,-1.46017549,-1.09010836
00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.20594055,-0.80272796,-0.75839062,-0.49368275,-0.54633356
002b83e4-5cce-414a-8570-871a50ae0964,-0.09575557,1.94424048,2.36244699,0.06538286,-0.21735976
005eec3f-d746-480f-ae2c-bc6f1151548d,0.46969787,1.44903056,0.90183852,-0.74191942,0.35272897
00b00ae4-229d-455d-84f1-d6face4876b1,0.06504951,1.74040081,-0.10669102,0.54529264,-0.45978148
00df2b78-66c8-4009-89e0-0e14e15e4208,-0.61841752,-0.61268158,0.52731819,-1.44535918,-0.99385284
0117a71e-3e6a-445e-be5a-b4e79e6feb78,-0.89414520,-0.67653201,0.40762381,-0.95054912,-0.33200922
0138d4f1-7fda-46f9-9ccb-8821c5b9ac4c,-1.43925631,5.00864980,4.03386331,-0.84505116,-0.20757524
01f84d32-7411-4d29-bdd3-92c513c11e85,-0.14792395,0.50888227,0.12748207,-0.30280759,-0.55733961
02435452-0025-44df-8305-06f6823226dc,0.32022362,-0.94007106,-0.52901242,-1.25120071,-1.35367625


SA_GUID__1,sens,prepare,respond,recover,enhExp
000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.43622697,-1.238116,-1.0974574,-2.8273850,-1.1550860
00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.23461573,-1.525224,-1.6983638,-0.9559314,-0.5788979
002b83e4-5cce-414a-8570-871a50ae0964,-0.10908713,3.694169,5.2905443,0.1266068,-0.2303147
005eec3f-d746-480f-ae2c-bc6f1151548d,0.53510759,2.753243,2.0196089,-1.4366008,0.3737558
00b00ae4-229d-455d-84f1-d6face4876b1,0.07411059,3.306863,-0.2389259,1.0558728,-0.4871867
00df2b78-66c8-4009-89e0-0e14e15e4208,-0.70453148,-1.164125,1.1808955,-2.7986957,-1.0530929


## Vulnerability

In [69]:
#sum the dimensions to create a total overall score of vulnerability
dimensionDatasetMergeZ$vuln <- rowSums(dimensionDatasetMergeZ[2:(ncol(dimensionDatasetMergeZ))], na.rm = TRUE)
head(dimensionDatasetMergeZ)

#scale the overall vulnerability
dimensionDatasetMergeZ$vuln <- scale(dimensionDatasetMergeZ$vuln)

#scale makes the data a list(?), so need to make it numeric to allow the shapefile to be created at export
dimensionDatasetMergeZ$vuln<- as.numeric(dimensionDatasetMergeZ$vuln)
head(dimensionDatasetMergeZ)

SA_GUID__1,sens,prepare,respond,recover,enhExp,vuln
000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.43622697,-1.238116,-1.0974574,-2.8273850,-1.1550860,-6.811885
00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.23461573,-1.525224,-1.6983638,-0.9559314,-0.5788979,-5.782561
002b83e4-5cce-414a-8570-871a50ae0964,-0.10908713,3.694169,5.2905443,0.1266068,-0.2303147,10.158981
005eec3f-d746-480f-ae2c-bc6f1151548d,0.53510759,2.753243,2.0196089,-1.4366008,0.3737558,4.916373
00b00ae4-229d-455d-84f1-d6face4876b1,0.07411059,3.306863,-0.2389259,1.0558728,-0.4871867,4.297493
00df2b78-66c8-4009-89e0-0e14e15e4208,-0.70453148,-1.164125,1.1808955,-2.7986957,-1.0530929,-5.257371


SA_GUID__1,sens,prepare,respond,recover,enhExp,vuln
000581a3-4ebd-4a74-b5f8-0bd78cd7ede5,0.43622697,-1.238116,-1.0974574,-2.8273850,-1.1550860,-0.9300676
00275248-3d78-4fdb-8e5c-8ff1f76aebea,-0.23461573,-1.525224,-1.6983638,-0.9559314,-0.5788979,-0.7895281
002b83e4-5cce-414a-8570-871a50ae0964,-0.10908713,3.694169,5.2905443,0.1266068,-0.2303147,1.3870623
005eec3f-d746-480f-ae2c-bc6f1151548d,0.53510759,2.753243,2.0196089,-1.4366008,0.3737558,0.6712589
00b00ae4-229d-455d-84f1-d6face4876b1,0.07411059,3.306863,-0.2389259,1.0558728,-0.4871867,0.5867596
00df2b78-66c8-4009-89e0-0e14e15e4208,-0.70453148,-1.164125,1.1808955,-2.7986957,-1.0530929,-0.7178208


## Merge

In [70]:
#merge all the indicators, domains, dimensions, and total vulnerability into one dataset
outputDataset <- merge(masterDataset, domainDatasetMerge, by = 'SA_GUID__1')
outputDataset <- merge(outputDataset, dimensionDatasetMergeZ, by = 'SA_GUID__1')
str(outputDataset)

'data.frame':	18919 obs. of  44 variables:
 $ SA_GUID__1           : chr  "000581a3-4ebd-4a74-b5f8-0bd78cd7ede5" "00275248-3d78-4fdb-8e5c-8ff1f76aebea" "002b83e4-5cce-414a-8570-871a50ae0964" "005eec3f-d746-480f-ae2c-bc6f1151548d" ...
 $ young                : num  3.388 -1.204 0.294 -1.334 1.713 ...
 $ old                  : num  -0.868 1.083 -0.656 1.455 -1.206 ...
 $ priSch               : num  1.714 -0.561 0.359 -1.358 2.912 ...
 $ volunteers           : num  -0.886 0.818 -0.524 0.275 -1.594 ...
 $ poorHealth           : num  -0.725 0.278 0.0765 0.4931 -0.3281 ...
 $ disability           : num  -0.9224 -0.6261 0.0677 0.4561 -0.0308 ...
 $ unemploy             : num  -0.6165 -0.2352 0.2224 -0.0766 0.7577 ...
 $ lowSkill             : num  -0.319 -0.718 -0.737 -1.916 1.527 ...
 $ farming              : num  -0.578 0.281 -0.113 -0.578 -0.578 ...
 $ rent                 : num  -0.629 -1.032 0.373 1.189 2.128 ...
 $ education            : num  -0.317 -0.699 0.152 -0.781 -0.541 ...
 $ eng

# Export

In [72]:
#set the export path
exportPath <- paste(exportDirectory, 'irelandVulnerability.csv', sep = '/')

#write the output to a csv
write.csv(outputDataset, exportPath, row.names = FALSE)

#get the small area spatial data
smallAreaShp <- st_read("../0_Scripts/downloads/SMALL_AREA_2022.shp")

#get just the GUID from the dataset
smallAreaShp <- smallAreaShp[c('SA_GUID__1')]

# #add the data to the spatial data using the GUID as the common field
smallAreaVulnerability <- merge(smallAreaShp, outputDataset, by ="SA_GUID__1")

#export to shp and geojson
st_write(smallAreaVulnerability, paste(exportDirectory, "/", "irelandVulnerability.geojson", sep = ""))
st_write(smallAreaVulnerability, paste(exportDirectory, "/", "irelandVulnerability.shp", sep = ""))

Reading layer `SMALL_AREA_2022' from data source `C:\Users\wcamaro\Documents\Reachout\ISVEHI-main_2024\0_Scripts\downloads\SMALL_AREA_2022.shp' using driver `ESRI Shapefile'
Simple feature collection with 18919 features and 28 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 417471.5 ymin: 519663.7 xmax: 734481.1 ymax: 966896.3
Projected CRS: IRENET95 / Irish Transverse Mercator
Layer irelandVulnerability in dataset ../2_OutputData/irelandVulnerability.geojson already exists:
use either append=TRUE to append to layer or append=FALSE to overwrite layer


ERROR: Error in CPL_write_ogr(obj, dsn, layer, driver, as.character(dataset_options), : Dataset already exists.



**END**